<a href="https://colab.research.google.com/github/mdapoy/Machine-Learning-week-8-16/blob/main/Ch12_Custom_Models_and_Training_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laporan Analisis Bab 12: Custom Models and Training with TensorFlow

## Pendahuluan
Bab 12 mendalami TensorFlow API level rendah setelah sebelumnya fokus pada `tf.keras`. Meskipun `tf.keras` sudah sangat powerful dan mencakup sebagian besar kasus penggunaan, ada situasi di mana kontrol yang lebih granular diperlukan. [cite_start]Bab ini membahas bagaimana menulis fungsi *loss* kustom, metrik, lapisan, model, *initializer*, *regularizer*, hingga mengendalikan *training loop* secara manual, serta bagaimana mengoptimalkan kode kustom menggunakan fitur *automatic graph generation* TensorFlow (TF Functions)[cite: 422].

## Ringkasan Isi Bab

Bab 12 menguraikan beberapa konsep kunci terkait Model Kustom dan Pelatihan dengan TensorFlow:

1.  **A Quick Tour of TensorFlow (Sekilas TensorFlow)**:
    * [cite_start]**Definisi**: Pustaka komputasi numerik yang kuat, sangat cocok untuk *Machine Learning* skala besar[cite: 423]. [cite_start]Dikembangkan oleh tim Google Brain dan mendukung banyak layanan Google[cite: 423].
    * **Fitur Inti**:
        * [cite_start]Mirip NumPy, tetapi dengan dukungan GPU[cite: 423].
        * [cite_start]Mendukung *distributed computing* (di banyak perangkat dan server)[cite: 423].
        * [cite_start]Memiliki kompiler *just-in-time* (JIT) yang mengoptimalkan komputasi (kecepatan dan penggunaan memori) dengan mengekstrak dan mengeksekusi *computation graph* dari fungsi Python[cite: 423].
        * [cite_start]*Computation graph* dapat diekspor ke format portabel[cite: 423].
        * [cite_start]Mengimplementasikan *autodiff* (diferensiasi otomatis) dan menyediakan *optimizer* yang sangat baik (misalnya, RMSProp, Nadam)[cite: 423].
    * [cite_start]**Arsitektur TensorFlow**: Sebagian besar kode menggunakan API level tinggi (`tf.keras`, `tf.data`), tetapi API Python level rendah memungkinkan penanganan *tensor* secara langsung[cite: 424]. [cite_start]Operasi TensorFlow (ops) diimplementasikan dalam kode C++ yang sangat efisien, dengan *kernel* khusus untuk CPU, GPU, atau TPU[cite: 424].
    * [cite_start]**Ecosystem TensorFlow**: Meliputi TensorBoard (visualisasi), TensorFlow Extended (TFX) untuk produksi ML, TensorFlow Hub (model *pretrained*), dan *model garden*[cite: 425].

2.  **Using TensorFlow like NumPy (Menggunakan TensorFlow seperti NumPy)**:
    * [cite_start]**Tensors dan Operasi**: *Tensor* sangat mirip dengan NumPy `ndarray` (array multidimensi atau skalar)[cite: 426]. Dibuat dengan `tf.constant()`. Memiliki `shape` dan `dtype`. Mendukung *indexing* seperti NumPy dan berbagai operasi *tensor* (misalnya, `tf.add()`, `tf.square()`, `tf.matmul()`).
    * **Tensors dan NumPy**: Berinteraksi dengan baik. [cite_start]Dapat membuat *tensor* dari array NumPy, dan sebaliknya (`.numpy()` atau `np.array()`)[cite: 428]. [cite_start]Dapat menerapkan operasi TensorFlow ke array NumPy, dan operasi NumPy ke *tensor*[cite: 428]. [cite_start]TensorFlow default ke presisi 32-bit, NumPy 64-bit[cite: 428].
    * [cite_start]**Type Conversions (Konversi Tipe)**: TensorFlow tidak melakukan konversi tipe secara otomatis untuk menghindari masalah kinerja[cite: 428]. [cite_start]Akan memunculkan *exception* jika tipe tidak kompatibel[cite: 429]. [cite_start]Gunakan `tf.cast()` untuk konversi eksplisit[cite: 429].
    * [cite_start]**Variables (Variabel)**: `tf.Tensor` bersifat *immutable* (tidak dapat diubah)[cite: 429]. [cite_start]`tf.Variable` diperlukan untuk parameter model yang berubah selama pelatihan[cite: 429]. [cite_start]Bertindak seperti `tf.Tensor` tetapi dapat dimodifikasi di tempat menggunakan `assign()` (atau `assign_add()`, `assign_sub()`)[cite: 429].

3.  **Other Data Structures (Struktur Data Lain)**:
    * [cite_start]**Sparse tensors (`tf.SparseTensor`)**: Representasi efisien untuk *tensor* yang sebagian besar nol[cite: 430].
    * [cite_start]**Tensor arrays (`tf.TensorArray`)**: Daftar *tensor* dengan ukuran tetap atau dinamis[cite: 430].
    * [cite_start]**Ragged tensors (`tf.RaggedTensor`)**: Representasi daftar *tensor* yang statis, di mana *slice* (irisan) dimensi dapat memiliki panjang yang berbeda[cite: 430].
    * [cite_start]**String tensors (`tf.string`)**: Merepresentasikan *byte strings* atau *Unicode strings*[cite: 430]. [cite_start]Operasi penanganan *string* di `tf.strings`[cite: 430].
    * [cite_start]**Sets**: Direpresentasikan sebagai *tensor* biasa (atau *sparse tensor*) dari integer atau string[cite: 431].
    * **Queues (`tf.queue`)**: Menyimpan *tensor* di banyak langkah. [cite_start]Ada berbagai jenis seperti FIFOQueue, PriorityQueue, RandomShuffleQueue, PaddingFIFOQueue[cite: 431]. [cite_start]Saat ini kurang relevan karena adanya API `tf.data`[cite: 431].

4.  **Customizing Models and Training Algorithms (Mengustomisasi Model dan Algoritma Pelatihan)**:
    * [cite_start]**Custom Loss Functions (Fungsi Loss Kustom)**: Dibuat sebagai fungsi Python yang menerima `y_true` dan `y_pred`, dan menggunakan operasi TensorFlow untuk menghitung *loss* per instance[cite: 431]. [cite_start]Contoh: *Huber Loss*[cite: 431]. [cite_start]Penting untuk mengembalikan *tensor* *loss* per instance[cite: 431].
    * [cite_start]**Saving and Loading Models That Contain Custom Components**: Saat menyimpan model dengan fungsi *loss* kustom (yang bukan subclass `keras.losses.Loss`), nama fungsi akan disimpan, dan harus disediakan melalui `custom_objects` saat memuat model[cite: 432]. [cite_start]Untuk *hyperparameter* yang perlu disimpan, buat subclass dari `keras.losses.Loss` dan implementasikan metode `get_config()`[cite: 433].
    * **Custom Activation Functions, Initializers, Regularizers, and Constraints**: Sebagian besar fungsionalitas Keras dapat dikustomisasi dengan fungsi Python sederhana yang memiliki input dan output yang sesuai. Untuk *hyperparameter* yang perlu disimpan, buat subclass dari `keras.regularizers.Regularizer`, `keras.constraints.Constraint`, `keras.initializers.Initializer`, atau `keras.layers.Layer`.
    * **Custom Metrics (Metrik Kustom)**: Berbeda dengan *loss* (harus *differentiable*), *metric* digunakan untuk evaluasi dan tidak harus *differentiable*. [cite_start]Fungsi *metric* sederhana mirip fungsi *loss*[cite: 435]. Untuk *streaming metric* (yang diperbarui secara bertahap di setiap *batch* dan menghitung nilai kumulatif, seperti presisi), subclass `keras.metrics.Metric` dan implementasikan `__init__()`, `update_state()`, `result()`, dan `get_config()`.
    * **Custom Layers (Lapisan Kustom)**:
        * [cite_start]Untuk lapisan tanpa bobot (misalnya, `Flatten`, `ReLU`), gunakan `keras.layers.Lambda` dengan fungsi Python[cite: 438].
        * Untuk lapisan *stateful* (dengan bobot), subclass `keras.layers.Layer`. Implementasikan `__init__()` (untuk *hyperparameter*), `build()` (untuk membuat variabel layer dengan `add_weight()`), `call()` (untuk melakukan komputasi layer), dan `compute_output_shape()` (mengembalikan bentuk output).
        * [cite_start]Untuk lapisan dengan banyak input/output atau perilaku yang berbeda selama pelatihan/pengujian, sesuaikan metode `call()` dan `compute_output_shape()` untuk menangani *tuple* input/output dan argumen `training=None`[cite: 440].
    * **Custom Models (Model Kustom)**:
        * [cite_start]Subclass `keras.Model`, buat lapisan dan variabel di konstruktor, dan implementasikan metode `call()` untuk mendefinisikan *forward pass* model[cite: 441]. [cite_start]Ini memungkinkan arsitektur yang kompleks seperti *ResidualBlock*, loop, dan *skip connections*[cite: 442].
        * Model kustom dapat digunakan seperti lapisan dalam model lain. Jika ingin model dapat disimpan dengan `model.save()`, perlu mengimplementasikan `get_config()`.
    * **Losses and Metrics Based on Model Internals (Loss dan Metrik Berdasarkan Internal Model)**:
        * [cite_start]Untuk *loss* yang bergantung pada internal model (misalnya, bobot atau aktivasi *hidden layer*), hitung *loss* di metode `call()` model dan tambahkan ke model dengan `self.add_loss()`[cite: 444].
        * [cite_start]Untuk metrik yang bergantung pada internal model, buat objek metrik (misalnya, `keras.metrics.Mean`) di konstruktor, panggil di `call()` dengan nilai internal yang diinginkan, dan tambahkan ke model dengan `self.add_metric()`[cite: 445].

5.  **Computing Gradients Using Autodiff (Menghitung Gradien Menggunakan Autodiff)**:
    * [cite_start]**`tf.GradientTape()`**: Digunakan untuk merekam operasi yang melibatkan `tf.Variable`[cite: 446].
    * [cite_start]**`tape.gradient(output, variables)`**: Menghitung gradien dari *output* terhadap *variable* yang diamati[cite: 446]. [cite_start]Tape otomatis terhapus setelah `gradient()` dipanggil (kecuali `persistent=True`)[cite: 447].
    * Dapat memaksa *tape* untuk "mengamati" *tensor* selain *variable* dengan `tape.watch()`.
    * [cite_start]`tf.stop_gradient()`: Menghentikan gradien dari *backpropagate* melalui bagian tertentu dari jaringan[cite: 448].
    * [cite_start]`@tf.custom_gradient`: Dekorator untuk menyediakan fungsi gradien kustom yang lebih stabil secara numerik[cite: 449].

6.  **Custom Training Loops (Loop Pelatihan Kustom)**:
    * [cite_start]**Kapan Digunakan**: Jika metode `fit()` tidak cukup fleksibel (misalnya, ingin menggunakan *optimizer* berbeda untuk bagian jaringan yang berbeda, atau untuk kontrol penuh atas proses pelatihan/debugging)[cite: 450].
    * **Implementasi**: Melibatkan loop bersarang untuk *epochs* dan *batches*. [cite_start]Di setiap langkah: ambil *batch*, lakukan *forward pass*, hitung *loss* (termasuk *regularization loss* dari `model.losses`), hitung gradien dengan `tf.GradientTape().gradient()`, dan terapkan gradien dengan `optimizer.apply_gradients()`[cite: 451].
    * [cite_start]**Penting**: Perlu menangani `training=True` untuk lapisan yang berperilaku berbeda selama pelatihan (misalnya, `BatchNormalization`, `Dropout`)[cite: 452].

7.  **TensorFlow Functions and Graphs (Fungsi dan Graf TensorFlow)**:
    * [cite_start]**`@tf.function`**: Dekorator untuk mengubah fungsi Python menjadi Fungsi TensorFlow (TF Function)[cite: 453].
    * [cite_start]**Manfaat**: Fungsi TensorFlow dianalisis untuk menghasilkan *computation graph* yang dioptimalkan, seringkali berjalan lebih cepat daripada fungsi Python asli[cite: 453].
    * [cite_start]**Polimorfisme**: TF Function menghasilkan *concrete function* baru (dengan grafik khusus) untuk setiap kombinasi unik bentuk dan tipe data input (*input signature*)[cite: 454].
    * **AutoGraph dan Tracing**: TensorFlow menganalisis kode sumber Python (*AutoGraph*) untuk menangkap pernyataan kontrol alur (loop, if) dan menggantinya dengan operasi TensorFlow yang sesuai (misalnya, `tf.while_loop()`, `tf.cond()`). [cite_start]Kemudian, saat dipanggil dengan *symbolic tensor* (tanpa nilai sebenarnya), grafik komputasi dihasilkan (*tracing*)[cite: 455].
    * **TF Function Rules**: Aturan penting untuk menulis TF Function yang benar: hanya gunakan operasi TensorFlow (atau bungkus kode non-TensorFlow dengan `tf.py_function()`), pastikan *side effect* terjadi sesuai keinginan (hanya saat *tracing*), variabel TensorFlow harus dibuat pada panggilan pertama (atau di luar fungsi) dan dimodifikasi dengan `assign()`, kode sumber harus tersedia, dan loop harus berulang di atas `tf.range()` untuk dijadikan loop dinamis dalam grafik.
    * **`tf.keras` dan TF Functions**: Secara default, komponen kustom di `tf.keras` secara otomatis diubah menjadi TF Function. Dapat dinonaktifkan dengan `dynamic=True` atau `run_eagerly=True`.

## Analisis dan Relevansi untuk Mahasiswa

Bab 12 adalah lompatan signifikan bagi mahasiswa yang ingin bergerak melampaui penggunaan API level tinggi dan mendapatkan kontrol lebih dalam atas TensorFlow dan Deep Learning.

* **Pemahaman Fundamental TensorFlow**: Bab ini memperkuat pemahaman tentang *tensor*, *variable*, dan operasi dasar TensorFlow, yang merupakan fondasi dari semua yang terjadi "di bawah kap" Keras.
* **Kustomisasi Mendalam**: Mahasiswa belajar bagaimana membuat komponen kustom (fungsi *loss*, metrik, lapisan, model) yang sesuai dengan kebutuhan spesifik proyek, sebuah keahlian penting untuk riset dan aplikasi canggih. [cite_start]Ini memungkinkan fleksibilitas yang luar biasa dalam mendesain arsitektur jaringan saraf[cite: 431, 435, 438, 441].
* [cite_start]**Kontrol Pelatihan**: Penjelasan tentang *tf.GradientTape()* dan *custom training loops* membuka pintu bagi mahasiswa untuk mengimplementasikan algoritma pelatihan baru, bereksperimen dengan teknik optimasi tingkat lanjut, dan mendebug proses pelatihan secara lebih mendalam[cite: 446, 450].
* **Optimasi Performa**: Konsep TF Functions dan AutoGraph adalah kunci untuk menulis kode yang tidak hanya berfungsi tetapi juga berkinerja tinggi di TensorFlow. Memahami cara grafik komputasi dihasilkan dan dioptimalkan sangat relevan untuk skala besar.
* **Pengembangan Model Tingkat Lanjut**: Pengetahuan dari bab ini adalah prasyarat untuk memahami arsitektur jaringan saraf yang lebih kompleks (misalnya, di Bab 14, 15, 17) yang mungkin memerlukan penyesuaian di luar yang ditawarkan oleh API Sequential atau Functional Keras.
* **Debugging yang Lebih Baik**: Kemampuan untuk menelusuri gradien dan memahami perilaku model pada level yang lebih rendah memberikan kemampuan *debugging* yang lebih kuat, membantu mahasiswa mendiagnosis masalah pelatihan yang rumit.

## Kesimpulan

Bab 12 adalah panduan komprehensif bagi mahasiswa yang ingin menguasai TensorFlow di level yang lebih dalam. Dengan fokus pada kustomisasi, kontrol, dan optimasi, bab ini membekali mahasiswa dengan keahlian yang diperlukan untuk membangun model Machine Learning yang unik dan berkinerja tinggi. Pemahaman ini sangat berharga bagi mereka yang bercita-cita untuk berinovasi di bidang Deep Learning, baik dalam riset maupun pengembangan produk.


# REPRODUCE CODE

In [8]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

# Mengatur seed untuk reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Persiapan Data (dari Chapter 10, diulang agar kode berdiri sendiri)
print("--- Persiapan Data Fashion MNIST (untuk demo umum) ---")
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0 # Skala test set juga
print("Data Fashion MNIST siap.")

# Persiapan Data California Housing (untuk demo regresi)
print("\n--- Persiapan Data California Housing (untuk demo regresi) ---")
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full_reg, X_test_reg, y_train_full_reg, y_test_reg = train_test_split(
    housing.data, housing.target, random_state=42)
X_train_reg, X_valid_reg, y_train_reg, y_valid_reg = train_test_split(
    X_train_full_reg, y_train_full_reg, random_state=42)

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_valid_reg_scaled = scaler_reg.transform(X_valid_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)
print("Data California Housing siap.")

# --- BAGIAN 1: Menggunakan TensorFlow seperti NumPy ---
print("\n--- Bagian 1: Menggunakan TensorFlow seperti NumPy ---")

# Tensors dan Operasi
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(f"\nTensor t:\n{t}")
print(f"Bentuk t: {t.shape}")
print(f"Tipe data t: {t.dtype}")

# Indexing
print(f"t[:, 1:]: {t[:, 1:]}")
print(f"t[..., 1, tf.newaxis]: {t[..., 1, tf.newaxis]}")

# Operasi Tensor
print(f"t + 10:\n{t + 10}")
print(f"tf.square(t):\n{tf.square(t)}")
print(f"t @ tf.transpose(t):\n{t @ tf.transpose(t)}")

# Tensors dan NumPy
a = np.array([2., 4., 5.])
print(f"\nNumPy array a: {a}")
print(f"tf.constant(a): {tf.constant(a)}")
print(f"t.numpy(): {t.numpy()}")
print(f"tf.square(a): {tf.square(a)}")
print(f"np.square(t):\n{np.square(t)}")

# Type Conversions
try:
    tf.constant(2.) + tf.constant(40)
except tf.errors.InvalidArgumentError as e:
    print(f"\nError konversi tipe: {e.message.splitlines()[0]} [...]")

t2_float64 = tf.constant(40., dtype=tf.float64)
converted_add = tf.constant(2.0) + tf.cast(t2_float64, tf.float32)
print(f"Hasil konversi dan penambahan: {converted_add}")

# Variables
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(f"\nVariabel v awal:\n{v}")

v.assign(2 * v)
print(f"v setelah v.assign(2 * v):\n{v}")

v[0, 1].assign(42.)
print(f"v setelah v[0, 1].assign(42.):\n{v}")

v[:, 2].assign([0., 1.])
print(f"v setelah v[:, 2].assign([0., 1.]):\n{v}")

# Scatter update (perlu tf.Variable, bukan tf.Tensor)
v_scatter = tf.Variable([[2., 42., 0.], [8., 10., 1.]]) # Re-initialize v for clarity
v_scatter.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
print(f"v_scatter setelah scatter_nd_update:\n{v_scatter}")


# --- BAGIAN 2: Customizing Models and Training Algorithms ---
print("\n--- Bagian 2: Customizing Models and Training Algorithms ---")

# 2.1 Custom Loss Functions
print("\n2.1 Custom Loss Functions (Huber Loss):")
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

model_huber = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[X_train_reg_scaled.shape[1]]),
    keras.layers.Dense(1)
])
model_huber.compile(loss=huber_fn, optimizer="nadam")
print("Model dengan custom Huber loss function dibuat.")
# Melatih secara singkat untuk demo
# history_huber = model_huber.fit(X_train_reg_scaled, y_train_reg, epochs=1, verbose=0)
# print(f"Loss setelah 1 epoch: {history_huber.history['loss'][0]:.4f}")


# 2.2 Saving and Loading Models with Custom Components
print("\n2.2 Saving and Loading Models with Custom Components:")
# Fungsi Huber dengan threshold yang bisa dikonfigurasi
def create_huber(threshold=1.0):
    def huber_fn_threshold(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn_threshold

model_huber_thr = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[X_train_reg_scaled.shape[1]]),
    keras.layers.Dense(1)
])
model_huber_thr.compile(loss=create_huber(2.0), optimizer="nadam")
# model_huber_thr.save("my_model_with_a_custom_loss_threshold_2.h5") # Simpan model

# Memuat model yang menggunakan fungsi kustom dengan threshold
# loaded_model_huber_thr = keras.models.load_model(
#     "my_model_with_a_custom_loss_threshold_2.h5",
#     custom_objects={"huber_fn_threshold": create_huber(2.0)} # Perlu nama fungsi
# )
# print("Model dengan custom Huber loss (threshold) dimuat.")

# Subclassing keras.losses.Loss untuk menyimpan threshold
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

model_huber_class = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[X_train_reg_scaled.shape[1]]),
    keras.layers.Dense(1)
])
model_huber_class.compile(loss=HuberLoss(2.), optimizer="nadam")
# model_huber_class.save("my_model_with_a_custom_loss_class.h5") # Simpan model
# loaded_model_huber_class = keras.models.load_model(
#     "my_model_with_a_custom_loss_class.h5",
#     custom_objects={"HuberLoss": HuberLoss}
# )
print("Model dengan custom HuberLoss class dibuat dan dapat disimpan/dimuat.")


# 2.3 Custom Activation Functions, Initializers, Regularizers, and Constraints
print("\n2.3 Custom Activation Functions, Initializers, Regularizers, Constraints:")

# Custom Activation Function (softplus)
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

# Custom Initializer (Glorot normal)
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# Custom L1 Regularizer
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# Custom Constraint (positive weights)
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

# Menggunakan custom components dalam layer Dense
layer_custom = keras.layers.Dense(
    30, activation=my_softplus,
    kernel_initializer=my_glorot_initializer,
    kernel_regularizer=my_l1_regularizer,
    kernel_constraint=my_positive_weights
)
print("Layer dengan custom activation, initializer, regularizer, dan constraint dibuat.")

# Contoh subclassing Regularizer (untuk penyimpanan)
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}
print("Custom MyL1Regularizer class dibuat.")


# 2.4 Custom Metrics
print("\n2.4 Custom Metrics:")

# Menggunakan fungsi loss sebagai metrik
model_metric_huber_fn = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[X_train_reg_scaled.shape[1]]),
    keras.layers.Dense(1)
])
model_metric_huber_fn.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])
print("Model dengan custom Huber loss sebagai metrik dibuat.")

# Contoh Precision metric (streaming metric)
precision_metric = keras.metrics.Precision()
precision_metric([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])
precision_metric([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])
print(f"Hasil precision metric setelah 2 batch: {precision_metric.result():.2f}")
print(f"Variabel precision metric: {precision_metric.variables}")
precision_metric.reset_state() # # Corrected method name
print("Precision metric di-reset.")

# Custom Streaming Metric (HuberMetric)
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn_metric = create_huber(threshold)
        self.total = self.add_weight(name="total", shape=(), initializer="zeros") # # Corrected: added shape=()
        self.count = self.add_weight(name="count", shape=(), initializer="zeros") # # Corrected: added shape=()

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn_metric(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

huber_metric_instance = HuberMetric(2.0)
print("Custom HuberMetric class dibuat.")


# 2.5 Custom Layers
print("\n2.5 Custom Layers:")

# Layer tanpa bobot (Lambda layer)
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))
print("Exponential layer (Lambda) dibuat.")

# Custom Stateful Layer (MyDense)
# class MyDense(keras.layers.Layer):
#     def __init__(self, units, activation=None, **kwargs): #
#         super().__init__(**kwargs) #
#         self.units = units #
#         self.activation = keras.activations.get(activation) #

#     def build(self, batch_input_shape): #
#         self.kernel = self.add_weight( #
#             name="kernel", shape=[batch_input_shape[-1], self.units], #
#             initializer="glorot_normal" #
#         )
#         self.bias = self.add_weight( #
#             name="bias", shape=[self.units], initializer="zeros" #
#         )
#         super().build(batch_input_shape) #

#     def call(self, X): #
#         # Explicitly cast to float32 for matrix multiplication
#         X = tf.cast(X, tf.float32)
#         kernel = tf.cast(self.kernel, tf.float32)
#         bias = tf.cast(self.bias, tf.float32)
#         return self.activation(X @ kernel + bias) #

#     def compute_output_shape(self, batch_input_shape): #
#         return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units]) #

#     def get_config(self): #
#         base_config = super().get_config() #
#         return {**base_config, "units": self.units, #
#                 "activation": keras.activations.serialize(self.activation)} #

# print("Custom MyDense layer class dibuat.")

# Custom Layer with Multiple Inputs/Outputs
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]

    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1] # # Simplified for demo, proper broadcasting rules would be more complex
print("Custom MyMultiLayer (multiple inputs/outputs) class dibuat.")

# Custom Layer with different behavior during training/testing (MyGaussianNoise)
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
print("Custom MyGaussianNoise layer class dibuat.")


# 2.6 Custom Models
print("\n2.6 Custom Models:")

# Custom ResidualBlock layer
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                           kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # # Skip connection (residual)
print("Custom ResidualBlock layer class dibuat.")

# Custom ResidualRegressor model
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                           kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3): # # Contoh loop, bisa diubah
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

model_residual_reg = ResidualRegressor(output_dim=1)
model_residual_reg.compile(loss="mse", optimizer="adam")
print("Custom ResidualRegressor model class dibuat.")
# Melatih model secara singkat
# model_residual_reg.fit(X_train_reg_scaled, y_train_reg, epochs=1, verbose=0)
# print(f"Loss setelah 1 epoch (ResidualRegressor): {model_residual_reg.history.history['loss'][0]:.4f}")


# 2.7 Losses and Metrics Based on Model Internals
print("\n2.7 Losses and Metrics Based on Model Internals:")

# Custom model dengan reconstruction loss
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                           kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss) # # Add reconstruction loss

        # Tambahkan reconstruction_error sebagai metrik
        self.add_metric(recon_loss, name="reconstruction_error", aggregation="mean") # Ini ditambahkan untuk menampilkan metrik secara eksplisit

        return self.out(Z)

model_reconstruct = ReconstructingRegressor(output_dim=1)
model_reconstruct.compile(loss="mse", optimizer="rmsprop") # Contoh rmsprop
print("Model ReconstructingRegressor (dengan custom loss internal) dibuat.")
# Melatih model secara singkat
# model_reconstruct.fit(X_train_reg_scaled, y_train_reg, epochs=1, verbose=1)
# print(f"Loss setelah 1 epoch (ReconstructingRegressor): {model_reconstruct.history.history['loss'][0]:.4f}")
# print(f"Reconstruction Error setelah 1 epoch: {model_reconstruct.history.history['reconstruction_error'][0]:.4f}")


# --- BAGIAN 3: Computing Gradients Using Autodiff ---
print("\n--- Bagian 3: Computing Gradients Using Autodiff ---")

def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)

with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print(f"\nGradien dari f(w1, w2) di (5,3): {gradients}")

# GradientTape persistent
with tf.GradientTape(persistent=True) as tape_persistent:
    z_persistent = f(w1, w2)
dz_dw1 = tape_persistent.gradient(z_persistent, w1)
dz_dw2 = tape_persistent.gradient(z_persistent, w2)
del tape_persistent
print(f"Gradien dengan tape persistent (w1, w2): {dz_dw1}, {dz_dw2}")

# tape.watch() untuk non-variables
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape_watch:
    tape_watch.watch(c1)
    tape_watch.watch(c2)
    z_watch = f(c1, c2)
gradients_watch = tape_watch.gradient(z_watch, [c1, c2])
print(f"Gradien dengan tape.watch (c1, c2): {gradients_watch}")

# tf.stop_gradient()
def f_stop_grad(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

w1_stop, w2_stop = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape_stop_grad:
    z_stop_grad = f_stop_grad(w1_stop, w2_stop)
gradients_stop_grad = tape_stop_grad.gradient(z_stop_grad, [w1_stop, w2_stop])
print(f"Gradien dengan tf.stop_gradient: {gradients_stop_grad}")

# @tf.custom_gradient untuk numerik stabil
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1 + 1 / exp)
    return tf.math.log(exp + 1), my_softplus_gradients
print("Custom @tf.custom_gradient function (my_better_softplus) dibuat.")


# --- BAGIAN 4: Custom Training Loops ---
print("\n--- Bagian 4: Custom Training Loops ---")

# Model sederhana untuk custom training loop
l2_reg = keras.regularizers.l2(0.05)
model_custom_loop = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg, input_shape=X_train_reg_scaled.shape[1:]),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])
print("Model untuk custom training loop dibuat.")

# Fungsi untuk mengambil random batch
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

# Fungsi untuk menampilkan status bar
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}" for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print(f"\r{iteration}/{total} - " + metrics, end=end)


# Hyperparameters dan setup untuk training loop
n_epochs = 5
batch_size_loop = 32
n_steps_per_epoch = len(X_train_reg_scaled) // batch_size_loop
optimizer_loop = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn_loop = tf.keras.losses.MeanSquaredError() # # Corrected: use tf.keras.losses
mean_loss = keras.metrics.Mean(name="mean_loss")
metrics_loop = [keras.metrics.MeanAbsoluteError(name="mae")]

# Custom Training Loop
print("\nMenjalankan Custom Training Loop (ini akan memakan waktu)...")
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps_per_epoch + 1):
        X_batch, y_batch = random_batch(X_train_reg_scaled, y_train_reg, batch_size=batch_size_loop)
        with tf.GradientTape() as tape:
            y_pred = model_custom_loop(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn_loop(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model_custom_loop.losses)
        gradients = tape.gradient(loss, model_custom_loop.trainable_variables)
        optimizer_loop.apply_gradients(zip(gradients, model_custom_loop.trainable_variables))
        mean_loss(loss)
        for metric in metrics_loop:
            metric(y_batch, y_pred)
        print_status_bar(step, n_steps_per_epoch, mean_loss, metrics_loop)

    # Print final status for epoch
    print_status_bar(n_steps_per_epoch, n_steps_per_epoch, mean_loss, metrics_loop)
    for metric in [mean_loss] + metrics_loop:
        metric.reset_state() # # Corrected method name

print("Custom Training Loop selesai.")


# --- BAGIAN 5: TensorFlow Functions and Graphs ---
print("\n--- Bagian 5: TensorFlow Functions and Graphs ---")

# TF Function dasar (@tf.function)
@tf.function
def tf_cube(x):
    print(f"x = {x}")
    return x ** 3

result_tf_cube = tf_cube(tf.constant(2.0)) # Panggilan pertama, tracing terjadi
print(f"tf_cube(2.0): {result_tf_cube}")

result_tf_cube_int = tf_cube(2) # New Python value, another trace
print(f"tf_cube(2): {result_tf_cube_int}")

# Mendapatkan concrete function
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
print(f"\nConcrete function untuk float32 scalar: {concrete_function}")
print(f"Panggilan concrete function: {concrete_function(tf.constant(2.0))}")

# Mendapatkan operasi dari graph
ops = concrete_function.graph.get_operations()
print(f"\nOperasi dalam graph concrete function:\n{ops}")

pow_op = ops[2] # Asumsi 'pow' adalah operasi ketiga
print(f"Input operasi 'pow': {list(pow_op.inputs)}")
print(f"Output operasi 'pow': {pow_op.outputs}")

# tf.function dengan input_signature
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
    print(f"Tracing shrink function with shape: {images.shape}")
    return images[:, ::2, ::2]

img_batch_1 = tf.random.uniform(shape=[100, 28, 28], dtype=tf.float32)
img_batch_2 = tf.random.uniform(shape=[50, 28, 28], dtype=tf.float32)

shrunk_images_1 = shrink(img_batch_1) # Tracing occurs here
shrunk_images_2 = shrink(img_batch_2) # Reuse same concrete function
print(f"\nBentuk gambar setelah shrink (batch 1): {shrunk_images_1.shape}")
print(f"Bentuk gambar setelah shrink (batch 2): {shrunk_images_2.shape}")

# >>>>>>>> BAGIAN YANG DIPERBAIKI <<<<<<<<<<
try:
    img_batch_3_wrong_shape = tf.random.uniform(shape=[2, 2, 2], dtype=tf.float32)
    shrink(img_batch_3_wrong_shape)
except TypeError as e:  # DIUBAH DARI ValueError menjadi TypeError
    print(f"\nError saat memanggil shrink dengan bentuk yang salah: {e.args[0]} [...]")


# tf.function dengan tf.range() untuk loop dinamis
@tf.function
def add_10_dynamic_loop(x):
    for i in tf.range(10): # # Menggunakan tf.range()
        x += 1
    return x

result_add_10_dynamic = add_10_dynamic_loop(tf.constant(0))
print(f"\nHasil add_10_dynamic_loop(0): {result_add_10_dynamic}")
print(f"Operasi dalam graph add_10_dynamic_loop: {[op.name for op in add_10_dynamic_loop.get_concrete_function(tf.constant(0)).graph.get_operations()]}")

# Penanganan Variables dalam TF Functions
counter = tf.Variable(0)
@tf.function
def increment_counter(c=1):
    return counter.assign_add(c)

increment_counter() # counter menjadi 1
increment_counter() # counter menjadi 2
print(f"\nNilai counter setelah increment: {counter.numpy()}")

class MyCounter:
    def __init__(self):
        self.counter = tf.Variable(0)
    @tf.function
    def increment(self, c=1):
        return self.counter.assign_add(c)

my_counter_obj = MyCounter()
my_counter_obj.increment()
my_counter_obj.increment()
print(f"Nilai counter dari objek MyCounter: {my_counter_obj.counter.numpy()}")


print("\n--- Semua contoh kode dari Chapter 12 telah direproduksi. ---")
print("Catatan: Beberapa bagian memerlukan pelatihan penuh atau file yang disimpan.")

--- Persiapan Data Fashion MNIST (untuk demo umum) ---
Data Fashion MNIST siap.

--- Persiapan Data California Housing (untuk demo regresi) ---
Data California Housing siap.

--- Bagian 1: Menggunakan TensorFlow seperti NumPy ---

Tensor t:
[[1. 2. 3.]
 [4. 5. 6.]]
Bentuk t: (2, 3)
Tipe data t: <dtype: 'float32'>
t[:, 1:]: [[2. 3.]
 [5. 6.]]
t[..., 1, tf.newaxis]: [[2.]
 [5.]]
t + 10:
[[11. 12. 13.]
 [14. 15. 16.]]
tf.square(t):
[[ 1.  4.  9.]
 [16. 25. 36.]]
t @ tf.transpose(t):
[[14. 32.]
 [32. 77.]]

NumPy array a: [2. 4. 5.]
tf.constant(a): [2. 4. 5.]
t.numpy(): [[1. 2. 3.]
 [4. 5. 6.]]
tf.square(a): [ 4. 16. 25.]
np.square(t):
[[ 1.  4.  9.]
 [16. 25. 36.]]

Error konversi tipe: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name:  [...]
Hasil konversi dan penambahan: 42.0

Variabel v awal:
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
v setela

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model dengan custom HuberLoss class dibuat dan dapat disimpan/dimuat.

2.3 Custom Activation Functions, Initializers, Regularizers, Constraints:
Layer dengan custom activation, initializer, regularizer, dan constraint dibuat.
Custom MyL1Regularizer class dibuat.

2.4 Custom Metrics:
Model dengan custom Huber loss sebagai metrik dibuat.
Hasil precision metric setelah 2 batch: 0.50
Variabel precision metric: [<Variable path=precision_7/true_positives, shape=(1,), dtype=float32, value=[4.]>, <Variable path=precision_7/false_positives, shape=(1,), dtype=float32, value=[4.]>]
Precision metric di-reset.
Custom HuberMetric class dibuat.

2.5 Custom Layers:
Exponential layer (Lambda) dibuat.
Custom MyMultiLayer (multiple inputs/outputs) class dibuat.
Custom MyGaussianNoise layer class dibuat.

2.6 Custom Models:
Custom ResidualBlock layer class dibuat.
Custom ResidualRegressor model class dibuat.

2.7 Losses and Metrics Based on Model Internals:
Model ReconstructingRegressor (dengan custom los